# Traffic Sign Classifier


### Load The Data

In [1]:
# Load pickled data
import pickle

training_file = 'train.p'
validation_file= 'valid.p'
testing_file =  'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

print('Data Loaded')

Data Loaded


### Summary of Data 

In [2]:
import numpy as np

n_train = X_train.shape[0]
n_validation = X_valid.shape[0]
n_test = X_test.shape[0]
image_shape = X_train.shape[1:3]
n_classes = len(np.unique(y_test))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Number of testing samples =",n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)


Number of training examples = 34799
Number of testing examples = 12630
Number of testing samples = 12630
Image data shape = (32, 32)
Number of classes = 43


### Plot Random Images From Training Set

In [ ]:
import matplotlib.pyplot as plt
import random
%matplotlib inline


#Plot images from the features set
image_num= random.randint(0,n_train)
print(X_train.shape)
image = X_train[image_num]

print(image.shape)
plt.imshow(image)

#imgplot = plt.imshow(image)

#plt.show()

### Preprocessing Functions

In [3]:
import imutils
from random import randint
import cv2
def rgb2gray(images):
    
    r, g, b = images[:,:,:,0], images[:,:,:,1], images[:,:,:,2]
    gray = 0.3 * r + 0.11* g + 0.59 * b
    gray = image_histogram_equalization(gray)
    return gray

"""
    Balances misrepresented classes by adding rotated versions of them to compensate
"""
def BalanceImageSet(X_set,y_set):
    
    unique, counts = np.unique(y_set, return_counts=True)
    count_dict = dict(zip(unique, counts))
    
    average_count = sum(count_dict[key] for key in count_dict) / len(count_dict)
    max_count = max(counts)
    
    new_images = []
    continue_balance = True
    
    while(continue_balance):
        new_values = []
        unique, counts = np.unique(y_set, return_counts=True)
        count_dict = dict(zip(unique, counts))
        
        continue_balance = False;
        for i in range(len(X_set)):
            if(count_dict[y_set[i]] < max_count/2):
                continue_balance = True
                
                #Rotate Image and Add
                rot_angle = randint(-10,10)
                rotated_image = imutils.rotate(X_set[i],rot_angle)
                new_images.append(rotated_image)
                new_values.append(y_set[i])
                
                #Translate Image and add
                translate_x,translate_y = randint(-5,5),randint(-5,5)
                translated_image = imutils.translate(X_set[i],translate_x,translate_y)
                new_images.append(translated_image)
                new_values.append(y_set[i])
                
        #Updated Y set so that we know how many we need to keep doing.         
        y_set = np.concatenate((y_set,np.array(new_values)))
                
    X_set = np.concatenate((X_set,np.array(new_images)))
        
    
    print(X_set.shape,y_set.shape)
   
    return X_set,y_set

def AugmentData(X_set, y_set):
    
    new_images = []
    new_values = []
    
    for i in range(len(X_set)):

        #Rotate Image and Add
        rot_angle = randint(-10,10)
        rotated_image = imutils.rotate(X_set[i],rot_angle)
        new_images.append(rotated_image)
        new_values.append(y_set[i])

        #Translate Image and add
        translate_x,translate_y = randint(-4,4),randint(-4,4)
        translated_image = imutils.translate(X_set[i],translate_x,translate_y)
        new_images.append(translated_image)
        new_values.append(y_set[i]) 

        #Change Luminescene and add
        new_image = X_set[i] - 10
        new_images.append(new_image)
        new_values.append(y_set[i])
        
        #Change Luminescene and add
        new_image = X_set[i] + 10
        new_images.append(new_image)
        new_values.append(y_set[i])

            
    augmented_y = np.array(new_values)
    augmented_X = np.array(new_images)
    
    return augmented_X, augmented_y

def hist_equal(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
    cl1 = clahe.apply(image)
    return cl1


### Balance Data

In [4]:
# Balance Data Types
X_train,y_train = BalanceImageSet(X_train,y_train)
print("Image balance complete")

X_train_extra,y_train_extra = AugmentData(X_train,y_train)
print("Data Augmented")

X_train = np.concatenate((X_train,X_train_extra))
y_train = np.concatenate((y_train,y_train_extra))

print(X_train.shape)

(63417, 32, 32, 3) (63417,)
Image balance complete
Data Augmented
(317085, 32, 32, 3)


### Model Architecture

In [5]:
#Example architecture
from tensorflow.contrib.layers import flatten
def LeNet(x,keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.01
    x_gray = tf.image.rgb_to_grayscale(x)
    
    # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x16.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 16), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(16))
    conv1   = tf.nn.conv2d(x_gray, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # Activation.
    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = 28x28x16. Output = 14x14x16.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    #Dropout
    conv1 = tf.nn.dropout(conv1,keep_prob)
    
    # Layer 2: Convolutional Input = 14x14x16 Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 16, 24), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(24))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # Activation.
    conv2 = tf.nn.relu(conv2)

    # Pooling. Input = 10x10x24. Output = 5x5x24.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    #Dropout
    conv2 = tf.nn.dropout(conv2,keep_prob)
    
    # Flatten. Input = 5x5x16. Output = 600.
    fc0   = flatten(conv2)
    
    # Layer 3: Fully Connected. Input = 600. Output = 240.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(600, 240), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(240))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # Activation.
    fc1    = tf.nn.relu(fc1)
    
    #Dropout
    fc1 = tf.nn.dropout(fc1,keep_prob)

    # Layer 4: Fully Connected. Input = 240. Output = 120.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(240, 120), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(120))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # Activation.
    fc2    = tf.nn.relu(fc2)
    
    #Dropout
    fc1 = tf.nn.dropout(fc1,keep_prob)

    # Layer 5: Fully Connected. Input = 120. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(120, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

### Build the Model

In [6]:
import tensorflow as tf
#Training Parameters
learning_rate = 0.0002
EPOCHS = 30
BATCH_SIZE = 128
KEEP_PROB = 0.6

#Parameters
keep_prob = tf.placeholder(tf.float32)

#Model
x = tf.placeholder(tf.float32,shape=(None,32,32,3))
logits=LeNet(x,keep_prob)

y = tf.placeholder(tf.int32,(None))
one_hot_y = tf.one_hot(y,43)


# Calculate the loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y,logits=logits)
loss = tf.reduce_mean(cross_entropy)

#Set up the optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

print("Model Built")

Model Built


### Model Evaluation

In [7]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
prediction = tf.argmax(logits,1)
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

softmax_logits = tf.nn.softmax(logits)
top_k = tf.nn.top_k(softmax_logits,k=5)

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y,keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Train the Model

In [8]:
from sklearn.utils import shuffle

with tf.Session() as sess:
    #Initiate variables
    #init = tf.global_variables_initializer()
    #sess.run(init)
    saver.restore(sess,'./lenet')
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x = X_train[offset:end]
            batch_y = y_train[offset:end]

            sess.run(training_op, feed_dict={x: batch_x, y: batch_y,keep_prob:KEEP_PROB})
            
        train_accuracy = evaluate(X_train,y_train)
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(train_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")
        

INFO:tensorflow:Restoring parameters from ./lenet
Training...

EPOCH 1 ...
Training Accuracy = 0.985
Validation Accuracy = 0.975

EPOCH 2 ...
Training Accuracy = 0.986
Validation Accuracy = 0.975

EPOCH 3 ...
Training Accuracy = 0.985
Validation Accuracy = 0.975

EPOCH 4 ...
Training Accuracy = 0.985
Validation Accuracy = 0.973

EPOCH 5 ...
Training Accuracy = 0.986
Validation Accuracy = 0.974

EPOCH 6 ...
Training Accuracy = 0.986
Validation Accuracy = 0.976

Model saved


### Test the Model on the Test Set

In [9]:
with tf.Session() as sess:
    saver.restore(sess, './lenet')

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./lenet
Test Accuracy = 0.960


## Step 3: Test Model on Images from the Internet


### Load and Output the Images

In [13]:
import matplotlib.image as mpimg
from PIL import Image
from scipy import ndimage
import matplotlib.pyplot as plt

size = [32,32]
images = []
y_internet = np.array([23,14,1,13,25])

for i in range(1,6):
    img_loc = 'germansigns/sign' + str(i) + '.jpg'
    img = Image.open(img_loc)
    img = img.resize(size,Image.ANTIALIAS)
    img = np.array(img)
    img = np.resize(img,(1,32,32,3))
    
    images.append(img)
    print(img.shape)

X_internet = np.vstack(images)

#Plot image
image = X_internet[4]
imgplot = plt.imshow(image)

#Convert to grayscale
#X_internet = np.resize(rgb2gray(X_internet),(X_internet.shape[0],32,32,1))


(1, 32, 32, 3)
(1, 32, 32, 3)
(1, 32, 32, 3)
(1, 32, 32, 3)
(1, 32, 32, 3)


### Predict the Sign Type for Each Image

In [14]:
with tf.Session() as sess:
    saver.restore(sess, './lenet')
    
    prediction = sess.run(prediction, feed_dict={x: X_internet, y: y_internet,keep_prob: 1.0})
    print("Prediction is " + str(prediction))
    
    validation_accuracy = evaluate(X_internet, y_internet)
    print("Accuracy is " + str(validation_accuracy))
    

INFO:tensorflow:Restoring parameters from ./lenet
Prediction is [23 14 23 13 22]
Accuracy is 0.600000023842


### Output Top 5 Softmax Probabilities For Each Image Found on the Web

In [15]:
with tf.Session() as sess:
    saver.restore(sess, './lenet')
    softmax = sess.run(top_k,feed_dict={x: X_internet, y: y_internet,keep_prob: 1.0})
    print("Top Softmax are: ")
    np.set_printoptions(precision=5,suppress=True)
    print(softmax[0])

INFO:tensorflow:Restoring parameters from ./lenet
Top Softmax are: 
[[ 0.99849  0.00074  0.00052  0.00021  0.00002]
 [ 0.99999  0.       0.       0.       0.     ]
 [ 0.20639  0.13894  0.11292  0.10266  0.04195]
 [ 0.99121  0.00194  0.00074  0.00061  0.00053]
 [ 0.59453  0.12667  0.08019  0.04463  0.04068]]
